In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
cd /content/gdrive/My\ Drive/Deep-Learning

/content/gdrive/My Drive/Deep-Learning


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform
import time, os, datetime
from Model import Residual_Unit
from Model import Attention_Block
from Model import AttentionResNet56_mini
from Model import AttentionResNet56

In [3]:
print("TF version: ",tf.__version__)
print("Keras version:",tf.keras.__version__)

TF version:  2.0.0
Keras version: 2.2.4-tf


In [0]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping,TensorBoard, LearningRateScheduler
from tensorflow.keras.models import load_model

In [5]:
# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train[:30000, :, :, :]
y_train = y_train[:30000]
x_val = x_train[-5000:, :, :, :]
y_val = y_train[-5000:]
# x_test = x_test[:10000, :, :, :]
# y_test = y_test[:10000:]

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, 10)
Y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (30000, 32, 32, 3)
y_train shape: (30000, 1)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


In [0]:
# define generators for training and validation data
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)

# compute quantities required for feature normalization
train_datagen.fit(x_train)
test_datagen.fit(x_val)
test_datagen.fit(x_test)

In [11]:
model = AttentionResNet56(shape=(32,32,3), in_channel=64, kernel_size=7, n_classes=10, dropout=0.4, regularization=0.0001)

def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 130:
        lr *= 0.5e-3
    elif epoch > 100:
        lr *= 1e-2
    elif epoch > 50:
        lr *= 1e-1
    print('Learning rate:', lr)
    return lr
lr_scheduler = LearningRateScheduler(lr_schedule)

# define loss, metrics, optimizer
optimizer = SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_230 (Conv2D)             (None, 32, 32, 64)   9472        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_164 (BatchN (None, 32, 32, 64)   256         conv2d_230[0][0]                 
__________________________________________________________________________________________________
activation_170 (Activation)     (None, 32, 32, 64)   0           batch_normalization_164[0][0]    
____________________________________________________________________________________________

In [0]:
batch_size = 128
epc = 150

start = time.time()

# training
train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
step_size_train = train_generator.n // train_generator.batch_size

# validation
val_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
step_size_val = val_generator.n // val_generator.batch_size

# test
test_generator = test_datagen.flow(x_test, y_test, batch_size=batch_size)
step_size_test = test_generator.n // test_generator.batch_size

# usefull callbacks
log_dir="Logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=7, min_lr=10e-7, verbose=1)
early_stopper = EarlyStopping(monitor='val_accuracy', patience=15, verbose=1)

model.fit_generator(train_generator,
                    steps_per_epoch = step_size_train,
                    epochs = epc,
                    validation_data = val_generator,
                    validation_steps = step_size_val,
                    callbacks=[tensorboard_callback, lr_reducer, lr_scheduler, early_stopper])

end = time.time()
print("Time taken by above cell is {}.".format((end-start)/60))

Learning rate: 0.001
Epoch 1/150
234/234 [==============================] - 273s 1s/step - loss: 2.6201 - accuracy: 0.2757 - val_loss: 1.8748 - val_accuracy: 0.2990
Learning rate: 0.001
Epoch 2/150
234/234 [==============================] - 271s 1s/step - loss: 1.8105 - accuracy: 0.3578 - val_loss: 1.6605 - val_accuracy: 0.3992
Learning rate: 0.001
Epoch 3/150
234/234 [==============================] - 270s 1s/step - loss: 1.6700 - accuracy: 0.4012 - val_loss: 1.5625 - val_accuracy: 0.4362
Learning rate: 0.001
Epoch 4/150
234/234 [==============================] - 270s 1s/step - loss: 1.5716 - accuracy: 0.4345 - val_loss: 1.4897 - val_accuracy: 0.4625
Learning rate: 0.001
Epoch 5/150
234/234 [==============================] - 269s 1s/step - loss: 1.5112 - accuracy: 0.4572 - val_loss: 1.4350 - val_accuracy: 0.4853
Learning rate: 0.001
Epoch 6/150
234/234 [==============================] - 267s 1s/step - loss: 1.4521 - accuracy: 0.4760 - val_loss: 1.4118 - val_accuracy: 0.4980
Learning r

In [12]:
# batch_size = 64
# epc = 50

# start = time.time()

# # training
# train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
# step_size_train = train_generator.n // train_generator.batch_size

# # validation
# val_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
# step_size_val = val_generator.n // val_generator.batch_size

# # test
# test_generator = test_datagen.flow(x_test, y_test, batch_size=batch_size)
# step_size_test = test_generator.n // test_generator.batch_size

# # usefull callbacks
# log_dir="Logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch = 100000000)
# lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=7, min_lr=10e-7, min_delta=0.01, verbose=1)
# early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=15, verbose=1)

# model.fit_generator(train_generator,
#                     steps_per_epoch = step_size_train,
#                     epochs = epc,
#                     validation_data = val_generator,
#                     validation_steps = step_size_val,
#                     callbacks=[tensorboard_callback, lr_reducer, early_stopper])

# end = time.time()
# print("Time taken by above cell is {}.".format((end-start)/60))

Epoch 1/50
78/78 [==============================] - 78s 994ms/step - loss: 2.6543 - accuracy: 0.2826 - val_loss: 2.6961 - val_accuracy: 0.1795
Epoch 2/50
78/78 [==============================] - 78s 997ms/step - loss: 2.5804 - accuracy: 0.2929 - val_loss: 2.6073 - val_accuracy: 0.1835
Epoch 3/50
78/78 [==============================] - 80s 1s/step - loss: 2.3796 - accuracy: 0.3096 - val_loss: 2.1113 - val_accuracy: 0.3071
Epoch 4/50
78/78 [==============================] - 81s 1s/step - loss: 2.3886 - accuracy: 0.3207 - val_loss: 2.5191 - val_accuracy: 0.2915
Epoch 5/50
78/78 [==============================] - 78s 1s/step - loss: 2.4780 - accuracy: 0.3217 - val_loss: 2.5335 - val_accuracy: 0.3053
Epoch 6/50
78/78 [==============================] - 78s 1s/step - loss: 2.2255 - accuracy: 0.3495 - val_loss: 1.9327 - val_accuracy: 0.3928
Epoch 7/50
78/78 [==============================] - 78s 1s/step - loss: 2.1138 - accuracy: 0.3600 - val_loss: 2.1967 - val_accuracy: 0.3854
Epoch 8/50
78/

In [1]:
# evaluation
val_scores = model.evaluate_generator(val_generator, verbose=0)
test_scores = model.evaluate_generator(test_generator, verbose=0)
print('validation loss:', val_scores[0])
print('validation accuracy:', val_scores[1])
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

NameError: ignored

In [0]:
model.save('Model56_paper.h5')  # 0.7418

In [18]:
ls

20191206-230234/


In [19]:
cd ..

/content/gdrive/My Drive/Deep-Learning
